### Import libraries

In [1]:
import QuantLib as ql
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

### Read the given data and clean up the format

In [2]:
data = pd.read_excel('FM5422 Project 1 Data.xlsx', skiprows=2)

In [3]:
# Print the table first to check what should we cleaning and preparing the data
data.head()

,Term,Unit,Maturity Date,Mid,Rate Type,Daycount,Freq,Unnamed: 7,Maturity Date.1,Discount
0,3,MO,2024-04-16,5.578140,Cash Rates,ACT/360,0,NaN,2024-04-16,0.985790
1,6,MO,2024-07-16,5.436519,Swap Rates,30I/360,2,NaN,2024-07-16,0.973225
2,1,YR,2025-01-16,4.925334,Swap Rates,30I/360,2,NaN,2025-01-16,0.952254
3,2,YR,2026-01-16,4.265320,Swap Rates,30I/360,2,NaN,2026-01-16,0.919041
4,3,YR,2027-01-19,4.003782,Swap Rates,30I/360,2,NaN,2027-01-19,0.887806


In [4]:
# Removing unnecessary rows and renaming columns for clarity
# Renaming the columns, dropping unnecessary columns and the first row which contains headers

cleaned_data = data

# Renaming the columns
cleaned_data.columns = ['Term', 'Unit', 'Maturity Date', 'Mid', 'Rate Type', 'Daycount', 'Freq', 'Column7', 'Maturity Date 2', 'Discount']

# dropping unnecessary columns and the first row which contains headers
cleaned_data = cleaned_data.drop(columns = ['Column7', 'Maturity Date 2', 'Discount'])

In [927]:
cleaned_data.head()

,Term,Unit,Maturity Date,Mid,Rate Type,Daycount,Freq
0,3,MO,2024-04-16,5.578140,Cash Rates,ACT/360,0
1,6,MO,2024-07-16,5.436519,Swap Rates,30I/360,2
2,1,YR,2025-01-16,4.925334,Swap Rates,30I/360,2
3,2,YR,2026-01-16,4.265320,Swap Rates,30I/360,2
4,3,YR,2027-01-19,4.003782,Swap Rates,30I/360,2


### Make sure to convert all the dates to QuantLib Date format

In [5]:
def convert_to_ql_date(date_str):
    return ql.Date(date_str.day, date_str.month, date_str.year)

cleaned_data['Maturity Date'] = cleaned_data['Maturity Date'].apply(convert_to_ql_date)

In [929]:
cleaned_data.head()

,Term,Unit,Maturity Date,Mid,Rate Type,Daycount,Freq
0,3,MO,"April 16th, 2024",5.578140,Cash Rates,ACT/360,0
1,6,MO,"July 16th, 2024",5.436519,Swap Rates,30I/360,2
2,1,YR,"January 16th, 2025",4.925334,Swap Rates,30I/360,2
3,2,YR,"January 16th, 2026",4.265320,Swap Rates,30I/360,2
4,3,YR,"January 19th, 2027",4.003782,Swap Rates,30I/360,2


### Calculate the discount factors for the first "Cash" assets. 

In [44]:
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

start_date = ql.Date(14, 1, 2024)
end_date = cleaned_data.loc[0,'Maturity Date']

day_counter = ql.Actual360()
day_count_3M = day_counter.dayCount(start_date, end_date)
day_count_3M

93

In [7]:
market_rate = cleaned_data.loc[0, 'Mid']
rate_3M = market_rate * day_count_3M / 360
rate_3M

1.4410195

In [8]:
discount_factor_3M = 1 / (1 + rate_3M / 100)
discount_factor_3M

0.9857945088968669

### Calculate the 6M discount factor from 6M swap rate

In [9]:
# follow the calculation in the excel sheet
start_date = ql.Date(14, 1, 2024)
end_date = cleaned_data.loc[1,'Maturity Date']

day_counter = ql.Thirty360(ql.Thirty360.USA)
day_count_6M = day_counter.dayCount(start_date, end_date)
day_count_6M

182

In [10]:
market_rate = cleaned_data.loc[1, 'Mid']

rate_6M = market_rate * day_count_6M / 360
rate_6M

2.7484621353906964

In [11]:
discount_factor_6M = 1 / (1 + rate_6M / 100)
discount_factor_6M

0.9732505764245009

### Calculate the 1Y discount factor from 1Y swap rate

In [936]:
# follow the calculation in the excel sheet, note that the 6M discount factor is already known, 
# so we only need to use a solver (root finder) to iteratively find the discount factor 
# such that (the discount cash flow) - 1 = 0 as shown in the sheet

In [12]:
market_rate = cleaned_data.loc[2, 'Mid']

rate_firsthalf = market_rate * day_count_6M / 360
rate_firsthalf

2.4900300703594755

In [14]:
start_date = cleaned_data.loc[1,'Maturity Date']
end_date = cleaned_data.loc[2,'Maturity Date']

day_counter = ql.Thirty360(ql.Thirty360.USA)
day_count_sechalf = day_counter.dayCount(start_date, end_date)
day_count_sechalf

180

In [15]:
rate_sechalf = market_rate * day_count_sechalf / 360
rate_sechalf

2.4626671025533273

In [16]:
def func(discount_factor_1yr):
    first_discount_cashflow = (rate_firsthalf / 100) * discount_factor_6M
    second_discount_cashflow = (1 + rate_sechalf / 100) * discount_factor_1yr
    return second_discount_cashflow + first_discount_cashflow - 1

initial_guess = [0.95]
discount_factor_1yr = fsolve(func, initial_guess)
discount_factor_1yr[0]

0.9523134577498885

### Start from the 2Y Swap rate, calculate the 2Y spot rate and the discount factor

In [941]:
#1. Calculate the 4 actual payment dates (use the modified following convention for handling weekend/holidays)

#2. Calculate the 4 coupons on the 4 dates (using the 30/360 day count convention)

#3. Guess an initial 2Y spot rate, which implies a 18M spot rate

#4. With the 6M, 1M, 18M (depends on the 2Y spot rate), 2Y spot rate (variable), we can calculate the 4 discount factors and the cashflows

#5. Use a solver to calculate the implied 2Y spot rate such the total cashflow is 0.  This would be the 2Y spot rate from the 2Y swap rate.

In [17]:
start_date = ql.Date(14, 1, 2024)
end_date = cleaned_data.loc[18, 'Maturity Date']
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
bussinessConvention = ql.ModifiedFollowing
terminationBussinessConvention = ql.ModifiedFollowing
dateGeneration = ql.DateGeneration.Backward
monthEnd = False

schedule = ql.Schedule (start_date, 
                        end_date, 
                        tenor, 
                        calendar, 
                        bussinessConvention, 
                        terminationBussinessConvention, 
                        dateGeneration, 
                        monthEnd)

trading_date = pd.DataFrame({'Maturity Date': list(schedule)})
trading_date.at[0, 'Maturity Date'] = start_date

In [29]:
trading_date

,Maturity Date
0,"January 14th, 2024"
1,"July 16th, 2024"
2,"January 16th, 2025"
3,"July 16th, 2025"
4,"January 16th, 2026"
...,...
96,"January 19th, 2072"
97,"July 18th, 2072"
98,"January 17th, 2073"
99,"July 17th, 2073"


In [30]:
df = pd.merge(trading_date, cleaned_data, on='Maturity Date', how='outer')

last_row = df.iloc[-1:]
df = df.iloc[:-1]
df = pd.concat([df.iloc[:1], last_row, df.iloc[1:]]).reset_index(drop=True)
df = df.reset_index(drop=True)

In [32]:
requirement = df.dropna().index
requirement

Int64Index([1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 25, 31, 41, 51, 61, 81,
            101],
           dtype='int64')

In [35]:
for i in df.index[1:]:
    df.loc[i, 'Tenor'] = ql.Thirty360(0).dayCount(start_date,df.loc[i,'Maturity Date'])/360
    df.loc[i,'Day Count']=ql.Thirty360(0).dayCount(df.loc[i-1,'Maturity Date'],df.loc[i,'Maturity Date'])

In [37]:
for i in df.index:
    if df.loc[i,'Rate Type']=='Cash Rates':
        daycount=ql.Actual360().dayCount(df.loc[i - 1, 'Maturity Date'], df.loc[i,'Maturity Date'])
        df.loc[i,'Day Count'] = daycount
        rate=df.loc[i,'Mid']*daycount/360
        discount=1/(1+rate/100)
        tenor=daycount/360
        spot=2*((1/discount)**(0.5/tenor)-1)
        df.loc[i,'Discount']=discount
        df.loc[i,'Spot']=spot

In [39]:
for i in df.index:
    if i == 2:
        daycount = df.loc[i,'Day Count']
        coupon=df.loc[i,'Mid'] * df.loc[i,'Tenor']
        discount=1/(1+coupon/100)
        spot=2*((1/discount)**(0.5/df.loc[i,'Tenor'])-1)
        df.loc[i,'Discount']=discount
        df.loc[i,'Spot']=spot

    if i == 3:
        spot = 2*((1/discount_factor_1yr[0])**(0.5/df.loc[i,'Tenor'])-1)
        df.loc[i,'Discount'] = discount_factor_1yr[0]
        df.loc[i,'Spot']=spot

In [41]:
current_index = 2 # Start at 6 month index [2] b/c it is the first swap rate Maturity date

def f(x):
    for n in range(i - current_index):
        df.loc[i - n,'Spot'] = (x * (df.loc[i - n,'Tenor'] - df.loc[current_index,'Tenor']) + df.loc[current_index,'Spot'] * (df.loc[i,'Tenor'] - df.loc[i - n,'Tenor'])) / (df.loc[i,'Tenor'] - df.loc[current_index,'Tenor'])
    df['Discount'] = 1 / (1 + df['Spot'] / 2) ** (df['Tenor']*2)
    df['DCF'] = df['Discount'] * df['Coupon']
    return 100-df.loc[1:i,'DCF'].sum()

for i in df.index[1:]:
    if df.loc[i,'Mid']>0:
        df.loc[:i,'Coupon'] = df['Day Count']/360*df.loc[i,'Mid']
        df.loc[i,'Coupon'] = df.loc[i,'Coupon']+100
        rate=fsolve(f,df.loc[i,'Mid']/100)
        current_index = i

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


### Iteratively repeat the process to solve for the 3Y-50Y spot rates and discount factors

In [43]:
df_data = cleaned_df[['Maturity Date', 'Spot', 'Discount']]
df_data

,Maturity Date,Spot,Discount
1,"April 16th, 2024",0.056157,0.985946
2,"July 16th, 2024",0.055027,0.972930
3,"January 16th, 2025",0.049490,0.952030
5,"January 16th, 2026",0.042594,0.918946
7,"January 19th, 2027",0.039889,0.887775
9,"January 18th, 2028",0.038719,0.857425
11,"January 16th, 2029",0.038213,0.827394
13,"January 16th, 2030",0.038027,0.797533
15,"January 16th, 2031",0.037985,0.768274
17,"January 16th, 2032",0.038027,0.739659


In [958]:
df_data.to_excel('output_result.xlsx', index=False)